In [0]:
csv_data = """id,name,category,price
1,Amit,Electronics,50000
2,Priya,Furniture,3000
3,Rahul,Stationery,200
4,Neha,Books,800
5,Karthik,Electronics,45000
"""

dbutils.fs.put("/dbfs/temp/products.csv", csv_data,overwrite=True)

Wrote 139 bytes.


True

In [0]:
df=spark.read.option("header",True).option("inferSchema",True).csv("/dbfs/temp/products.csv")
df.show()

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  2|  Priya|  Furniture| 3000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
+---+-------+-----------+-----+



In [0]:
df.write.format("delta").mode("overwrite").save("/temp/delta/products")

In [0]:
df_delta=spark.read.format("delta").load("/temp/delta/products")
df_delta.show()

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  2|  Priya|  Furniture| 3000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
+---+-------+-----------+-----+



In [0]:
from delta.tables import DeltaTable

delta_table=DeltaTable.forPath(spark,"/temp/delta/products")

delta_table.update(
    condition="id=2",
    set={"price":"3500"}
)

delta_table.toDF().show()



+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
|  2|  Priya|  Furniture| 3500|
+---+-------+-----------+-----+



In [0]:
new_data = [
    (2, "Priya", "Furniture", 4000),     # Update
    (6, "Sneha", "Kitchen", 1200)        # Insert
]

updates_df=spark.createDataFrame(new_data,["id","name","category","price"])

delta_table.alias("target").merge(
    updates_df.alias("source"),
    "target.id = source.id"
).whenMatchedUpdateAll()\
.whenNotMatchedInsertAll()\
.execute()

delta_table.toDF().show()

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
|  2|  Priya|  Furniture| 4000|
|  6|  Sneha|    Kitchen| 1200|
+---+-------+-----------+-----+



In [0]:
delta_table.toDF().show()

pervious_version=spark.read.format("delta").option("versionAsOf",0).load("/temp/delta/products")
pervious_version.show()

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
|  2|  Priya|  Furniture| 4000|
|  6|  Sneha|    Kitchen| 1200|
+---+-------+-----------+-----+

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  1|   Amit|Electronics|50000|
|  2|  Priya|  Furniture| 3000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
|  5|Karthik|Electronics|45000|
+---+-------+-----------+-----+



In [0]:
df.write.format("delta").mode("overwrite").partitionBy("category").save("/temp/delta/products_partitioned")


In [0]:
spark.read.format("delta").load("/temp/delta/products_partitioned").show()

+---+-------+-----------+-----+
| id|   name|   category|price|
+---+-------+-----------+-----+
|  5|Karthik|Electronics|45000|
|  1|   Amit|Electronics|50000|
|  2|  Priya|  Furniture| 3000|
|  3|  Rahul| Stationery|  200|
|  4|   Neha|      Books|  800|
+---+-------+-----------+-----+

